In [1]:
import json
import os
import datetime
import time as Time
from posixpath import commonpath, splitext
from bs4.element import NavigableString
from dateutil.parser import parse
from sys import excepthook
from bs4 import BeautifulSoup
import requests
import uuid
import re
from requests import exceptions 

Next decide which urls to scarpe and put them into an array so you can iterate through them

In [2]:

#URL's to be scrapped
url1 = 'https://3dbrewing.com/events/'
url2 = ''
url3 = ''
url4 = ''
url5 = ''
url_array = [url1] 

Create a function that will iterate through the url_array and scrape webpage

In [3]:
#Properties
venue_name = '3 Daughters Brewing'
band_name = ''
date_string = ''
shows = []
venue_array = []

In [32]:
def scape_urls(arr):
    for i in arr:
        response = requests.get(i).text
        soup = BeautifulSoup(response, 'lxml')
        
        for single_Show in soup.find_all('div', class_='col event-post-container'):

        #Grab its information and organize it
            try:
                
                #Date
                date_Div = single_Show.find('h3', class_ = 'second-headline').text
                remove_EndTime = date_Div.split('-', 2)
                cleaned = remove_EndTime[0] + ' ' + remove_EndTime[1]

                date = parse(cleaned)
                date_string = '{:%b %d, %Y %-I:%M%p}'.format(date)

                #Band
                clean_Name = single_Show.find('h3', class_ = 'headline').text
                band_name = " ".join(clean_Name.split())

                showDict = {}
                showDict['band'] = band_name
                showDict['dateString'] = date_string
                
                
                #Filter Data
                filter_array = [
                    'TRIVIA', 'Trivia', 'Food', 'Blood Drive', 'Total Wine', 'Beer', 'ABC', 'Bingo' 
                ]

                if any([x in band_name for x in filter_array]):
                    continue
                
                shows.append(showDict)
                

            except AttributeError as ex:
                print('Error', ex)
    
    #Export as JSON
    venDict = {}
    venDict['venueName'] = venue_name
    venDict['shows'] = shows

    venue_array = [venDict]
    finalDict = {}
    finalDict['venue'] = venue_array

    #Save To json file
    save_path = '/Users/nathanhedgeman/Documents/Scrappers/Show Data'
    file_name = venue_name + '.json'
    complete_name = os.path.join(save_path, file_name)

    file = open(complete_name, 'w')
    file.write(json.dumps(finalDict, indent = 2))
    file.close()
    print("Complete!")

In [33]:
scape_urls(url_array)

Complete!
